In [224]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import ast
folder_path = "data"
single_data = "data/participant_3007.csv"
df_list = []

# Iterate through all CSV files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".csv"):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file)
        print(f"Reading: {file}")
        try:
            df_individual = pd.read_csv(file_path, on_bad_lines='skip')
            df_list.append(df_individual)
        except Exception as e:
            print(f"Error reading {file}: {e}")

df = pd.concat(df_list, ignore_index=True)
df_individual = pd.read_csv(single_data, on_bad_lines='skip')
df['ID'] = df['ID'].ffill()
df['ID'] = df['ID'].astype(int)
df['PayoutDistNum'] = df['PayoutDistNum'].ffill()
df['PayoutDistNum'] = df['PayoutDistNum'].astype(int)

trial_df = df[df['TrialType']=='pirate_recog'].dropna(axis=1,how='all')
trial_df['reward'] = trial_df['reward'].astype(int)

trial_df.groupby('ID')['reward'].mean()

Reading: participant_1111.csv
Reading: participant_3007.csv
Reading: participant_3014.csv
Reading: participant_3025.csv


ID
3007    0.948276
3014    0.932203
3025    0.813559
Name: reward, dtype: float64

In [53]:
# Step 1: Convert string to actual list
location_list = ast.literal_eval(df_individual['contextOrder'][0])

# Step 2: Get unique items
unique_locations = list(set(location_list))


print(unique_locations)

['stonepine_forest', 'icecap_mountain', 'coral_beach', 'greenrock_mountain', 'sunleaf_forest', 'driftwood_beach']


In [182]:
df_individual['reward'] = pd.to_numeric(df_individual['reward'], errors='coerce')
df_individual[df_individual['TrialType']=='pirate_recog'].dropna(axis=1, how='all').reset_index(drop='index')


,TrialType,TimeElapsed,key_press,RT,context,choice,reward,confidence,TimeInBlock
0,pirate_recog,1994.329320,5.0,1.493831,6,sure_old,1.0,sure,2.612281
1,pirate_recog,2034.948889,6.0,2.202731,5,unsure_old,1.0,unsure,3.320870
2,pirate_recog,2043.445671,6.0,1.658560,3,unsure_old,1.0,unsure,2.778579
3,pirate_recog,2059.703735,5.0,1.725569,5,sure_old,1.0,sure,2.843975
4,pirate_recog,2067.139963,5.0,1.638367,4,sure_old,1.0,sure,2.756509
5,pirate_recog,2100.407909,5.0,1.272488,3,sure_old,1.0,sure,2.392487
6,pirate_recog,2108.640989,6.0,1.927404,1,unsure_old,1.0,unsure,3.043458
7,pirate_recog,2115.623048,5.0,1.469493,6,sure_old,1.0,sure,2.584989
8,pirate_recog,2130.171623,8.0,1.369872,NaN,sure_new,1.0,sure,2.487032
9,pirate_recog,2137.169626,7.0,1.568687,NaN,unsure_new,1.0,unsure,2.685051


In [214]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.tools import add_constant
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
pt2_df = df_individual[df_individual['TrialType'].str.contains("pt2_pirate")].dropna(axis=1, how='all').reset_index(drop='index')
import pandas as pd
def numeric_choice(row):
    if row['choice'] == "red_pirate":
        row['choice_N'] = 0
    if row['choice'] == "white_pirate":
        row['choice_N'] = 1
    if row['choice'] == "black_pirate":
        row['choice_N'] = 2
# Example: df has columns 'choice' and 'reward'
# Create lagged reward columns
pt2_df['reward_t-1'] = pt2_df['reward'].shift(1)
pt2_df['reward_t-2'] = pt2_df['reward'].shift(2)
pt2_df['reward_t-3'] = pt2_df['reward'].shift(3)

# Drop rows with NaN values due to shifting
pt2_df = pt2_df.dropna().reset_index(drop=True)

import statsmodels.api as sm

X = pt2_df[['reward_t-1', 'reward_t-2', 'reward_t-3']]
X = sm.add_constant(X)
y = pt2_df['reward']
print(X.dtypes)
print(y.unique())
model = sm.Logit(y, X).fit()
print(model.summary())
# Drop NaNs from lagging
pt2_df = pt2_df.dropna(subset=['reward_t-1', 'reward_t-2', 'reward_t-3', 'choice'])

# Encode categorical choice into numbers
le = LabelEncoder()
pt2_df['choice_encoded'] = le.fit_transform(pt2_df['choice'])  # pirate1 → 0, pirate2 → 1, etc.

# Define predictors and outcome
X = pt2_df[['reward_t-1', 'reward_t-2', 'reward_t-3']]
y = pt2_df['choice_encoded']

from sklearn.linear_model import LogisticRegression

# Multinomial logistic regression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500)
model.fit(X, y)

# Get coefficients
import numpy as np
for i, pirate in enumerate(le.classes_):
    print(f"Coefficients for choosing {pirate}: {1/(1 + np.exp(-model.coef_[i]))}")

pt2_df['choice_prev'] = pt2_df['choice'].shift(1)
pt2_df['reward_prev'] = pt2_df['reward'].shift(1)

# Create column: was the previously rewarded pirate chosen again?
pt2_df['repeat_rewarded'] = ((pt2_df['choice'] == pt2_df['choice_prev']) & (pt2_df['reward_prev'] == 1)).astype(int)
import statsmodels.api as sm

X = sm.add_constant(pt2_df['reward_prev'])  # or use more lags if you like
y = pt2_df['repeat_rewarded']

model = sm.Logit(y, X, missing='drop').fit()
# print(model.summary())

pd.crosstab(pt2_df['reward_prev'], pt2_df['repeat_rewarded'])
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Rescale features (optional but often helpful)
X_scaled = StandardScaler().fit_transform(pt2_df[['reward_prev']].dropna())
y = pt2_df['repeat_rewarded'][0:126].dropna()

# Logistic regression with L2 penalty (default)
model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)
model.fit(X_scaled, y)

print("Coefficient:", model.coef_)
print("Intercept:", model.intercept_)

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

# Make sure the lags exist
pt2_df['choice_prev'] = pt2_df['choice'].shift(1)
pt2_df['reward_t-1'] = pt2_df['reward'].shift(1)
pt2_df['reward_t-2'] = pt2_df['reward'].shift(2)
pt2_df['reward_t-3'] = pt2_df['reward'].shift(3)

# Define repeat_rewarded: 1 if they chose the same pirate as last trial and it was previously rewarded
pt2_df['repeat_rewarded'] = ((pt2_df['choice'] == pt2_df['choice_prev']) & (pt2_df['reward_t-1'] == 1)).astype(int)

# Drop any rows with NaNs
df_model = pt2_df.dropna(subset=['reward_t-1', 'reward_t-2', 'reward_t-3', 'repeat_rewarded'])

# Define features and outcome
X = df_model[['reward_t-1', 'reward_t-2', 'reward_t-3']]
y = df_model['repeat_rewarded']

# Optional: scale predictors
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit multinomial logistic regression with L2 penalty
model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)
model.fit(X_scaled, y)

# Output coefficients
coef_labels = ['reward_t-1', 'reward_t-2', 'reward_t-3']
for i, coef in enumerate(model.coef_[0]):
    print(f"{coef_labels[i]} coefficient: {coef:.4f} (odds ratio ≈ {1 / (1 + np.exp(-coef)):.2f})")

print(f"Intercept: {model.intercept_[0]:.4f}")

const         float64
reward_t-1    float64
reward_t-2    float64
reward_t-3    float64
dtype: object
[0. 1.]
Optimization terminated successfully.
         Current function value: 0.635291
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 reward   No. Observations:                  127
Model:                          Logit   Df Residuals:                      123
Method:                           MLE   Df Model:                            3
Date:                Sat, 19 Jul 2025   Pseudo R-squ.:                 0.02961
Time:                        10:48:59   Log-Likelihood:                -80.682
converged:                       True   LL-Null:                       -83.144
Covariance Type:            nonrobust   LLR p-value:                    0.1775
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

/Users/rohinpalsule/Downloads/cogs108/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/rohinpalsule/Downloads/cogs108/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


reward_t-1 coefficient: 2.6191 (odds ratio ≈ 0.93)
reward_t-2 coefficient: 0.0780 (odds ratio ≈ 0.52)
reward_t-3 coefficient: 0.1334 (odds ratio ≈ 0.53)
Intercept: -2.1395


In [163]:
target_rows = df_individual[df_individual['TrialType'] == 'pirate_recog'].index + 1
# Filter only valid indices (i.e., not beyond the end of the DataFrame)
valid_indices = target_rows[target_rows < len(df)]

# Select the rows at those indices
result_df = df_individual.loc[valid_indices].reset_index(drop=True)
result_df[result_df['TrialType'].str.contains("pt2")].dropna(axis=1,how='all')

,TrialType,TimeElapsed,key_press,RT,context,reward_prob_red,reward_prob_white,reward_prob_black,choice,reward,TimeInBlock
0,pt2_pirate_13,2000.188126,3.0,2.288763,blank,0.682,0.261,0.292,black_pirate,0,5.598565
1,pt2_pirate_20,2040.397223,3.0,1.837995,blank,0.714,0.377,0.061,black_pirate,0,5.150828
2,pt2_pirate_21,2047.815512,3.0,0.763364,blank,0.781,0.458,0.115,black_pirate,1,4.070837
3,pt2_pirate_24,2064.080419,2.0,0.806959,blank,0.845,0.207,0.074,white_pirate,0,4.104979
4,pt2_pirate_25,2071.117042,2.0,0.408654,blank,0.661,0.250,0.092,white_pirate,1,3.707483
5,pt2_pirate_31,2105.287646,1.0,1.294519,blank,0.542,0.300,0.142,red_pirate,0,4.603850
6,pt2_pirate_32,2112.728329,1.0,0.501517,blank,0.461,0.369,0.057,red_pirate,0,3.810491
7,pt2_pirate_33,2119.543741,1.0,0.346634,blank,0.507,0.505,0.135,red_pirate,0,3.645355
8,pt2_pirate_36,2134.168512,2.0,0.419335,blank,0.742,0.266,0.055,white_pirate,0,3.717232
9,pt2_pirate_37,2141.268946,2.0,0.518941,blank,0.739,0.380,0.145,white_pirate,0,3.817202


In [68]:
# Find index of the first occurrence of 'pirate_1'
start_idx = df_individual[df_individual['TrialType'] == 'pt2_pirate_2'].index[0]

# Find index of the first occurrence of 'Instruction_44'
end_idx = df_individual[df_individual['TrialType'] == 'Instruction_50'].index[0]

# Slice the dataframe from start to end (inclusive)
sub_df = df_individual.loc[start_idx:end_idx-1].reset_index(drop=True)
sub_df.dropna(axis=1,how='all')

,TrialType,TimeElapsed,key_press,RT,context,reward_prob_red,reward_prob_white,reward_prob_black,choice,reward,confidence,TimeInBlock
0,pt2_pirate_2,1951.360322,1.0,1.279492,blank,0.325,0.131,0.553,red_pirate,0,NaN,4.572023
1,pt2_pirate_3,1955.271303,1.0,0.286130,blank,0.337,0.258,0.700,red_pirate,1,NaN,3.585580
2,pt2_pirate_4,1959.180297,1.0,0.349435,blank,0.294,0.129,0.669,red_pirate,1,NaN,3.655093
3,pt2_pirate_5,1963.143188,1.0,0.373476,blank,0.200,0.057,0.646,red_pirate,0,NaN,3.673647
4,pt2_pirate_6,1967.725121,2.0,1.021062,blank,0.249,0.069,0.778,white_pirate,1,NaN,4.326738
...,...,...,...,...,...,...,...,...,...,...,...,...
185,pt2_pirate_128,2667.550165,2.0,0.597125,blank,0.182,0.680,0.332,white_pirate,0,NaN,3.907523
186,pt2_pirate_129,2671.748532,2.0,0.520330,blank,0.146,0.676,0.268,white_pirate,0,NaN,3.821361
187,pt2_pirate_130,2676.454090,2.0,0.982806,blank,0.050,0.670,0.345,white_pirate,0,NaN,4.295712
188,pt2_pirate_131,2680.516653,2.0,0.345424,blank,0.175,0.420,0.359,white_pirate,0,NaN,3.653414


In [136]:
best_pirate_df = df_individual[df_individual['TrialType'] == 'best_pirate'].dropna(axis=1,how='all').reset_index(drop='index')

In [155]:
import ast
import pandas as pd

# Parse context order
chosen_pirates = best_pirate_df['choice'].dropna().values
context_order = ast.literal_eval(df_individual['contextOrder'].dropna().values[0])
unique_contexts = pd.Series(context_order).unique()

# Initialize list to store correctness
correct_flags = []
first_flags = []
for i, c in enumerate(unique_contexts):
    # Filter relevant rows
    df_context = df_individual[
        (df_individual['TrialType'].str.startswith("pirate_")) &
        (df_individual['context'] == c)
    ].dropna(axis=1, how='all')

    # Assign index column
    df_context['index'] = ['first' if j < 10 else 'main' for j in range(len(df_context))]

    # Group and compute means
    group_means = df_context.drop(columns='probe') \
        .groupby('index')[['reward_prob_red', 'reward_prob_white', 'reward_prob_black']] \
        .mean().reset_index()

    # Get max-probability color in "main" trials
    main_row = group_means[group_means['index'] == 'main'][['reward_prob_red', 'reward_prob_white', 'reward_prob_black']]
    max_col = main_row.idxmax(axis=1).values[0].replace("reward_prob_", "")

    # Get actual choice
    chose = chosen_pirates[i].replace("_pirate", "")

    # Append correctness flag
    correct_flags.append(1 if max_col == chose else 0)

    print(f"Context: {c} | Chose: {chose} | Correct: {max_col == chose} | Best Color: {max_col}")


    # Get max-probability color in "main" trials
    main_row = group_means[group_means['index'] == 'first'][['reward_prob_red', 'reward_prob_white', 'reward_prob_black']]
    next_col = main_row.idxmax(axis=1).values[0].replace("reward_prob_", "")

    # Get actual choice
    chose = chosen_pirates[i].replace("_pirate", "")

    # Append correctness flag
    first_flags.append(1 if next_col == chose else 0)

    print(f"Context: {c} | Chose: {chose} | Correct (first ten): {next_col == chose} | Best Color: {next_col}")

# Assign the full list to the DataFrame
best_pirate_df['correct'] = correct_flags
best_pirate_df['first_correct'] = first_flags

Context: driftwood_beach | Chose: red | Correct: True | Best Color: red
Context: driftwood_beach | Chose: red | Correct (first ten): False | Best Color: black
Context: sunleaf_forest | Chose: white | Correct: True | Best Color: white
Context: sunleaf_forest | Chose: white | Correct (first ten): False | Best Color: red
Context: icecap_mountain | Chose: red | Correct: False | Best Color: black
Context: icecap_mountain | Chose: red | Correct (first ten): False | Best Color: white
Context: coral_beach | Chose: black | Correct: False | Best Color: white
Context: coral_beach | Chose: black | Correct (first ten): True | Best Color: black
Context: stonepine_forest | Chose: red | Correct: True | Best Color: red
Context: stonepine_forest | Chose: red | Correct (first ten): False | Best Color: white
Context: greenrock_mountain | Chose: white | Correct: False | Best Color: black
Context: greenrock_mountain | Chose: white | Correct (first ten): False | Best Color: red


In [156]:
best_pirate_df

,TrialType,TimeElapsed,key_press,RT,context,choice,TimeInBlock,correct,first_correct
0,best_pirate,2897.314222,1.0,5.412254,driftwood_beach,red_pirate,6.457768,1,0
1,best_pirate,2902.253513,2.0,2.939885,sunleaf_forest,white_pirate,3.992941,1,0
2,best_pirate,2905.148844,1.0,0.924853,icecap_mountain,red_pirate,1.975256,0,0
3,best_pirate,2907.539325,3.0,0.418973,coral_beach,black_pirate,1.467796,0,1
4,best_pirate,2909.981892,1.0,0.467849,stonepine_forest,red_pirate,1.516283,1,0
5,best_pirate,2912.175758,2.0,0.252005,greenrock_mountain,white_pirate,1.299392,0,0


In [150]:
max_col==chose

True

In [146]:
chosen_pirates[0].replace("_pirate","")

'red'